In [1]:
import dagshub
dagshub.init(repo_owner='sahilbhardwaj23', repo_name='Youtube-comment-Sentiment-Analysis', mlflow=True)

Accessing as sahilbhardwaj23

Initialized MLflow to track repo "sahilbhardwaj23/Youtube-comment-Sentiment-Analysis"

Repository sahilbhardwaj23/Youtube-comment-Sentiment-Analysis initialized!

In [2]:
import mlflow
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow")

In [4]:
from mlflow.tracking import MlflowClient
import mlflow

In [5]:
def set_or_create_experiment(experiment_name):
    client = MlflowClient()
    experiment = client.get_experiment_by_name(experiment_name)

    # Check if the experiment exists and is active
    if experiment:
        if experiment.lifecycle_stage == "active":
            print(f"Using existing experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            mlflow.set_experiment(experiment_name)
        else:
            # Restore if it's deleted
            print(f"Restoring deleted experiment '{experiment_name}' (ID: {experiment.experiment_id})")
            client.restore_experiment(experiment.experiment_id)
            mlflow.set_experiment(experiment_name)
    else:
        # Create a new experiment if it doesn't exist
        print(f"Creating new experiment '{experiment_name}'")
        mlflow.set_experiment(experiment_name)



In [6]:
# Set or create an experiment
set_or_create_experiment("ML Algos with HP Tuning")

Creating new experiment 'ML Algos with HP Tuning'


2024/11/02 22:48:09 INFO mlflow.tracking.fluent: Experiment with name 'ML Algos with HP Tuning' does not exist. Creating a new experiment.


In [7]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from lightgbm import LGBMClassifier
import mlflow
import mlflow.sklearn
import optuna

e:\Projects\Youtube Comment Analysis\Youtube commnent Plugin\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
df = pd.read_csv('reddit_preprocessing.csv').dropna()
df.shape

(36662, 2)

In [9]:
# Step 1: Remap the class labels from [-1, 0, 1] to [2, 0, 1]
df['category'] = df['category'].map({-1: 2, 0: 0, 1: 1})

# Step 2: Remove rows where the target labels (category) are NaN
df = df.dropna(subset=['category'])

# Step 3: TF-IDF vectorizer setup
ngram_range = (1, 3)  # Trigram
max_features = 1000  # Set max_features to 1000
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)
X = vectorizer.fit_transform(df['clean_comment'])
y = df['category']

# Step 4: Apply SMOTE to handle class imbalance
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Step 5: Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42, stratify=y_resampled)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for LightGBM
def objective_lightgbm(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_test, model.fit(X_train, y_train).predict(X_test))


# Step 7: Run Optuna for LightGBM, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_lightgbm, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = LGBMClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "LightGBM"
    log_mlflow("LightGBM", best_model, X_train, X_test, y_train, y_test)

# Run the experiment for LightGBM
run_optuna_experiment()


[I 2024-11-02 22:49:43,355] A new study created in memory with name: no-name-046e37cb-277e-40df-bc54-cdee214821ef


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.064410 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99025
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 971
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-11-02 22:49:46,907] Trial 0 finished with value: 0.606002959205242 and parameters: {'n_estimators': 52, 'learning_rate': 0.004970455935845532, 'max_depth': 7}. Best is trial 0 with value: 0.606002959205242.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.062126 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 99025
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 971
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-11-02 22:49:55,249] Trial 1 finished with value: 0.5894102726696259 and parameters: {'n_estimators': 157, 'learning_rate': 0.0001414123950173089, 'max_depth': 7}. Best is trial 0 with value: 0.606002959205242.


...

[I 2024-11-02 22:53:02,319] Trial 28 finished with value: 0.7673853307968717 and parameters: {'n_estimators': 288, 'learning_rate': 0.03591704523244787, 'max_depth': 5}. Best is trial 12 with value: 0.7892623124075249.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.058971 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99025
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 971
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612


[I 2024-11-02 22:53:09,300] Trial 29 finished with value: 0.6643415768336504 and parameters: {'n_estimators': 133, 'learning_rate': 0.009314337722495167, 'max_depth': 7}. Best is trial 12 with value: 0.7892623124075249.


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.061808 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 99025
[LightGBM] [Info] Number of data points in the train set: 37848, number of used features: 971
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Info] Start training from score -1.098612
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further s

2024/11/02 22:53:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2024/11/02 22:53:39 INFO mlflow.tracking._tracking_service.client: 🏃 View run LightGBM_SMOTE_TFIDF_Trigrams at: https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow/#/experiments/7/runs/f55fad8830df45b2808befad9e9dda6f.
2024/11/02 22:53:39 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/sahilbhardwaj23/Youtube-comment-Sentiment-Analysis.mlflow/#/experiments/7.
